<a href="https://colab.research.google.com/github/TomoharuKurosu/Tomoharu_DS2/blob/main/%E3%83%AA%E3%82%B9%E3%83%88%E4%BD%9C%E6%88%90_%E8%A7%A3%E8%AA%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%
'1.要対協_要対協_被虐待数_昨年度累計_昨年差分 <= 0.5'
'5.家児相.csv_家児相_ケース区分=虐待_昨年度累計回数 <= 0.5'
'3.生活保護.csv_被保護世帯員個人情報一覧_世帯被生保延べ人数 <= 1.5'
'3.ひとり親医療受給者.csv_ひとり親医療受給者_受給回数_昨年差分 <= 0.5'
'1.要対協_要対協_被虐待数_昨年度累計 <= 0.5'
'5.家児相.csv_家児相_ケース区分=相談_昨年度累計回数 <= 0.5'
'2.出生～４か月健診.csv_出生～４か月健診_総合判定=要観察_フラグ <= 0.5'
'3.こども医療受給者.csv_こども医療受給者_受給回数 <= 0.5'
'4.世帯人数.csv_世帯人数_3歳未満人数 <= 0.5'
# %%
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print("{}return {}".format(indent, tree_.value[node]))

    recurse(0, 1)
tree_to_code(model, _X_train.columns.to_list())
# %%
def segment(df):
  if df["1.要対協_要対協_被虐待数_昨年度累計_昨年差分"] <= 0.5:
    if df["5.家児相.csv_家児相_ケース区分=虐待_昨年度累計回数"] <= 0.5:
      if df["3.生活保護.csv_被保護世帯員個人情報一覧_世帯被生保延べ人数"] <= 1.5:
        if df["3.ひとり親医療受給者.csv_ひとり親医療受給者_受給回数_昨年差分"] <= 0.5:
          if df["1.要対協_要対協_被虐待数_昨年度累計"] <= 0.5:
            if df["5.家児相.csv_家児相_ケース区分=相談_昨年度累計回数"] <= 0.5:
              if df["2.出生～４か月健診.csv_出生～４か月健診_総合判定=要観察_フラグ"] <= 0.5:
                if df["3.こども医療受給者.csv_こども医療受給者_受給回数"] <= 0.5:
                  return 19
                else:
                  return 18
              else:
                if df["4.世帯人数.csv_世帯人数_3歳未満人数"] <= 0.5:
                  return 16
                else:
                  return 15
            else:
              return 12
          else:
            return 10
        else:
          return 8
      else:
        return 6
    else:
      return 4
  else:
    return 2

# %%
proba = model.predict_proba(X_ptest.loc[:, lambda df: ~ df.columns.str.contains("虐待（疑いを含む）|3歳６か月健診_方法理由=43.0|方法=保健師フォロー|当てはまるもの=感情的に叩いた")])[:,1]
# print(roc_auc_score(y_true=y_ptest, y_score=proba))
valid = (
    ptest
    .iloc[:, :6]
    .assign(決定木_被虐待確率 = proba)
)
# %%
base = (
    valid
    .rename(columns={"1.要対協_要対協_被虐待フラグ": "要対協_2023年の9月までのデータ"})
    [["年度", "宛名番号", "要対協_2023年の9月までのデータ", "決定木_被虐待確率"]]
    .assign(セグメントID = ptest.apply(lambda r: segment(r), axis=1))
)
base
# %%
(
    base
    .groupby("セグメントID")
    ["決定木_被虐待確率"]
    .describe()
)
# %%
gakkou = (
    pd
    .read_csv("../data_raw/学校教育室_3ファイル/学齢簿.csv")
    # .columns.to_list()
    .rename(columns={"児生＿宛名番号":"宛名番号"})
    .set_index("宛名番号")
    .filter(regex="校コード")
    .reset_index()
    .pipe(check_df)
)

# %%
yotaikyo = (
    pd
    .read_csv("../data_clns+/1.要対協_目的変数.csv")
    .loc[lambda df: df["要対協_被虐待フラグ"]==1]
    .loc[lambda df: df["年度"] < 2023]
    .drop(columns="要対協_被虐待フラグ")
    .astype({"年度":str})
    .groupby("宛名番号", as_index=False)
    ["年度"]
    # .apply(lambda x: ",".join(x))
    .agg({
        "最新_要対協年度": lambda x: max(x.astype(int)),
        "最新_要対協からの経過年数": lambda x: 2023 - max(x.astype(int)),
        "過去_要対協年度一覧": lambda x: ",".join(x),
    })
)
yotaikyo
# %%
koumu = (
    pd
    .read_csv("../receive/20231219_校務支援/校務支援.csv")
    [["宛名番号", "学校名"]]
    .rename(columns={"学校名": "校務支援_学校名"})
)
koumu
# %%
prod = (
    base
    .merge(
        yotaikyo
        , on="宛名番号"
        , how="left"
    )
    .merge(
        gakkou
        , on="宛名番号"
        , how="left"
    )
    .merge(
        koumu
        , on="宛名番号"
        , how="left"
    )
)
prod
# %%
(
  prod
  .to_excel("../doc/20240116_和泉市身体的虐待スコアリスト.xlsx", index=False)
)
# %%
# 要対協.csv  要対協_被虐待数_昨年度累計_昨年差分
# 要対協.csv  要対協_被虐待数_昨年度累計
# 家児相.csv  家児相_ケース区分=虐待_昨年度累計回数
# 家児相.csv  家児相_ケース区分=相談_昨年度累計回数
# 生活保護.csv  被保護世帯員個人情報一覧_世帯被生保延べ人数
# 出生～４か月健診.csv  出生～４か月健診_総合判定=要観察_フラグ
# ひとり親医療受給者.csv  ひとり親医療受給者_受給回数_昨年差分
# こども医療受給者.csv  こども医療受給者_受給回数
# 世帯構成.csv  世帯人数_3歳未満人数
import pandas as pd
# %%
(
  pd
  .read_excel("../doc/20240116_和泉市身体的虐待スコアリスト.xlsx")
  .groupby("セグメントID")
  .drop_duplicates()
  .agg({
    "宛名番号": "nunique",
    "要対協_2023年の9月までのデータ": "sum",
    "決定木_被虐待確率": "mean",
  })
  # .to_excel("./2023_valid.xlsx")
)
# %%
(
  pd
  .read_excel("../doc/20240116_和泉市身体的虐待スコアリスト.xlsx")
  .loc[lambda df: df["宛名番号"].duplicated(keep=False)]
)
pd.DataFrame().drop_duplicates